### --- Day 13: Distress Signal ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/13

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 13);
WriteLines(inputLines);

Loading puzzle file: Day13.txt
Total lines: 449
Max line length: 223

[[],[[3,[0,3,0,7,9],3],[0,1]],[[],4],[[[8,5]]],[9,5,5,1]]
[[[[3,1,9,3],[1,1,1,9],1,[4,9,3,2],[9,1,8,7]],[1],[[4,2,10,2,5],[]],[[9,7,1],5,2,2,10],2],[],[[10,10,[],0]]]

[[[5],7,0]]
[[3,8,5],[9,[[4,10,0,4],[2,8,8,8,1],[2,6],[9,5,1,0,5]],10,6,5],[]]


In [4]:
string[] testInputLines = 
[
    "[1,1,3,1,1]",
    "[1,1,5,1,1]",
    "",
    "[[1],[2,3,4]]",
    "[[1],4]",
    "",
    "[9]",
    "[[8,7,6]]",
    "",
    "[[4,4],4,4]",
    "[[4,4],4,4,4]",
    "",
    "[7,7,7,7]",
    "[7,7,7]",
    "",
    "[]",
    "[3]",
    "",
    "[[[]]]",
    "[[]]",
    "",
    "[1,[2,[3,[4,[5,6,7]]]],8,9]",
    "[1,[2,[3,[4,[5,6,0]]]],8,9]",
];

In [5]:
interface Packet {}

record Scalar(int Value) : Packet {}

class PacketList(IList<Packet> items) : Packet, IEnumerable<Packet>
{
    public PacketList() : this([]) {}

    public IEnumerator<Packet> GetEnumerator() => items.GetEnumerator();

    public void Add(Packet item) => items.Add(item);

    public int Count => items.Count;
    
    IEnumerator IEnumerable.GetEnumerator() => this.GetEnumerator();

    // The following required for part 2

    public Packet this[int i] => items[i];
}

I managed to trip myself up here and had to peek at Peter Norvig's solution to resolve my mistake. I mistakenly interpreted the puzzle input as needing to work through all pairs `a, b` ensuring `a <= b`. What the puzzle input actually describes is more like an alphabetical sort: if the first pair is shown to be in order (or not), the two lists are sorted. Only if `a == b` do we need to compare the next pair.

First, let's model the data structures and the comparison...

In [6]:
using Ternary = int;
const Ternary Neutral = 0;
public static bool IsOrdered(this Ternary t) => t < 0;

static Ternary CheckOrder(Packet left, Packet right)
{
    return (left, right) switch {
        (Scalar l, Scalar r) => CheckOrderScalarScalar(l, r),
        (Scalar l, PacketList r) => CheckOrderScalarPacketList(l, r),
        (PacketList l, Scalar r) => CheckOrderPacketListScalar(l, r),
        (PacketList l, PacketList r) => CheckOrderPacketList2(l, r),
        _ => throw new Exception($"Unexpected comparison!")
    };
}

static Ternary CheckOrderScalarScalar(Scalar left, Scalar right)
    => left.Value - right.Value;

static Ternary CheckOrderScalarPacketList(Scalar left, PacketList right)
    => CheckOrderPacketList2([left], right);

static Ternary CheckOrderPacketListScalar(PacketList left, Scalar right)
    => CheckOrderPacketList2(left, [right]);

static Ternary CheckOrderPacketList2(PacketList left, PacketList right)
{
    var hasDiff = left.Zip(right)
                    .Select(lr => CheckOrder(lr.First, lr.Second))
                    .FirstOrDefault(x => x != Neutral);

    if (hasDiff is not Neutral)
    {
        return hasDiff;
    }

    var leftFinishesFirst = left.Count - right.Count;
    return leftFinishesFirst;
}

Now for the parsing...

In [7]:
(PacketList, string) ParsePacketList(string chars)
{
    List<Packet> result = new();
    
    // Expect [
    chars = chars[1..];

    while (chars[0] is not ']')
    {
        // Expect comma-separated packets or end of the list

        var (packet, remain) = ParsePacket(chars);
        result.Add(packet);

        chars = remain[0] switch {
            ',' => remain[1..], // another packet
            ']' => remain, // end of list
            _ => throw new Exception($"Unexpected list item: {remain[0]}")
        };
    }

    return (new(result), chars[1..]);
}

(Packet, string) ParseScalar(string chars)
{
    int result = -1;
    int charEnd = 1;
    while (int.TryParse(chars[0..charEnd], out var found))
    {
        result = found;
        charEnd += 1;
    }
    charEnd -= 1;

    return (new Scalar(result), chars[charEnd..]);
}

(Packet, string) ParsePacket(string chars)
{
    (Packet, string) result = chars[0] switch {
        '[' => ParsePacketList(chars),
        _ => ParseScalar(chars),
    };
    return result;
}

In [8]:
int FindPairsInOrder(string[] inputLines)
{
    var pairs = inputLines.SeparateBy(l => l is "").ToArray();

    return pairs.Index()
        .Where(x => PairIsOrdered(x.Item))
        .Select(x => x.Index + 1).Sum();
}

bool PairIsOrdered(string[] pair)
{
    var ((left, _), (right, _)) = pair.Select(ParsePacket).ToArray();
    return CheckOrder(left, right).IsOrdered();
}

In [9]:
// In the above example, the pairs in the right order are 1, 2, 4, and 6; the
// sum of these indices is 13.

var testAnswer = FindPairsInOrder(testInputLines);
Console.WriteLine(testAnswer);

13


In [10]:
// Determine which pairs of packets are already in the right order. What is the
// sum of the indices of those pairs?

var part1Answer = FindPairsInOrder(inputLines);
Console.WriteLine(part1Answer);

5852


In [11]:
// 5852 is correct!
Ensure(5852, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/13

Ok, I think we can leverage .NET's sorting mechanisms here, if we can shoehorn our comparison functions into the IComparer interface

In [13]:
class PacketComparer : IComparer<Packet>
{
    public int Compare(Packet left, Packet right) => CheckOrder(left, right);
}

In [14]:
string[] dividers = 
[
    "[[2]]",
    "[[6]]",
];

int SortAndDivide(string[] inputLines)
{
    var fullSet = inputLines.Where(line => line is not "").Concat(dividers);

    var packetSet = fullSet.Select(ParsePacket).Select(x => x.Item1).ToList();
    packetSet.Sort(new PacketComparer());

    var indexProduct = 1;
    foreach ((var i, PacketList p) in packetSet.Cast<PacketList>().Index())
    {
        if (p is [PacketList pl] && pl is [Scalar s] && s.Value is 2 or 6)
        {
            indexProduct *= i + 1;
        }
    }

    return indexProduct;
}

In [15]:
// In this example, the divider packets are 10th and 14th, and so the decoder
// key is 140.

var part2TestAnswer = SortAndDivide(testInputLines);
Console.WriteLine(part2TestAnswer);

140


In [16]:
// Organize all of the packets into the correct order. What is the decoder key
// for the distress signal?

var part2Answer = SortAndDivide(inputLines);
Console.WriteLine(part2Answer);

24190


In [17]:
// 24190 is correct!
Ensure(24190, part2Answer);